# TYC 70 nucleus segmentation pipeline

In [1]:
#used csbdeep
from stardist.models import StarDist2D
import numpy as np
import cv2
import os
import tensorflow as tf
import time
from scipy.optimize import linear_sum_assignment
from skimage.measure import regionprops, label
import random
from csbdeep.utils import normalize

# Test if GPU is available the code below is 2 ways to check if the CPU is available if the list is empty or the other line prints False your GPU is not available
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

#tf.test.is_gpu_available(cuda_only=True, min_cuda_compute_capability=(7,0))
tf.config.list_physical_devices('GPU')

# run this command if ImportError: DLL load failed: The specified module could not be found. for opencv
# pip install opencv-contrib-python

[]

In [2]:
from stardist.models import StarDist2D 
# prints a list of available models 
StarDist2D.from_pretrained() 
# creates a pretrained model
model = StarDist2D.from_pretrained('2D_versatile_fluo')

There are 4 registered models for 'StarDist2D':

Name                  Alias(es)
────                  ─────────
'2D_versatile_fluo'   'Versatile (fluorescent nuclei)'
'2D_versatile_he'     'Versatile (H&E nuclei)'
'2D_paper_dsb2018'    'DSB 2018 (from StarDist 2D paper)'
'2D_demo'             None
Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


## Other datasets accuracy

### Creation of our segmentation

In [3]:
# --- Paths ---
base_raw = r"\\store\department\gene\chien_data\Lab\Data_and_Analysis\Wilco_van_Nes\Cell_tracking_challenge_datasets\Fluo-N2DL-HeLa\Fluo-N2DL-HeLa\02"
out_nucleus = r"\\store\department\gene\chien_data\Lab\Data_and_Analysis\Wilco_van_Nes\Cell_tracking_challenge_datasets\Fluo-N2DL-HeLa\Fluo-N2DL-HeLa\02_stardist"

os.makedirs(out_nucleus, exist_ok=True)


# --- Parameters ---
prob_thresh = 0.15
frames_to_process = range(0, 84)

def normalize_percentile(img, pmin=1, pmax=99.8, axis=(0, 1)):
    """
    Normalize image based on percentiles.
    """
    mi = np.percentile(img, pmin, axis=axis, keepdims=True)
    ma = np.percentile(img, pmax, axis=axis, keepdims=True)
    norm_img = (img - mi) / (ma - mi)
    norm_img = np.clip(norm_img, 0, 1)
    return norm_img

In [ ]:
for frame in frames_to_process:
    fname_raw = f"t{frame:03d}.tif"
    
    fpath_raw = os.path.join(base_raw, fname_raw)

    if not os.path.exists(fpath_raw):
        print(f"Skipping frame {frame:03d}: file not found.")
        continue
 
    # --- nucleus segmentation ---
    t = time.time()

    fluorescent_raw_img = cv2.imread(fpath_raw, -1)

    # fluorescent_norm_img = normalize_percentile(fluorescent_raw_img, 1, 99.8, axis=(0, 1))

    # fl_labels, polys = model.predict_instances(fluorescent_norm_img, prob_thresh = prob_thresh)
    fl_labels, polys = model.predict_instances(fluorescent_raw_img, prob_thresh = prob_thresh)

    # --- Save nucleus segmentation ---
    out1 = os.path.join(out_nucleus, fname_raw)
    cv2.imwrite(out1, fl_labels.astype(np.uint16))
    
    elapsed = time.time() - t
    print(f'Segmenting frame-{frame:05d} took {int(elapsed)} seconds.')

    

Segmenting frame-00000 took 0 seconds.
Segmenting frame-00001 took 0 seconds.
Segmenting frame-00002 took 0 seconds.
Segmenting frame-00003 took 0 seconds.
Segmenting frame-00004 took 0 seconds.
Segmenting frame-00005 took 0 seconds.
Segmenting frame-00006 took 0 seconds.
Segmenting frame-00007 took 0 seconds.
Segmenting frame-00008 took 0 seconds.
Segmenting frame-00009 took 0 seconds.
Segmenting frame-00010 took 0 seconds.
Segmenting frame-00011 took 1 seconds.
Segmenting frame-00012 took 0 seconds.
Segmenting frame-00013 took 0 seconds.
Segmenting frame-00014 took 0 seconds.
Segmenting frame-00015 took 1 seconds.
Segmenting frame-00016 took 0 seconds.
Segmenting frame-00017 took 0 seconds.
Segmenting frame-00018 took 0 seconds.
Segmenting frame-00019 took 0 seconds.
Segmenting frame-00020 took 0 seconds.
Segmenting frame-00021 took 1 seconds.
Segmenting frame-00022 took 0 seconds.
Segmenting frame-00023 took 1 seconds.
Segmenting frame-00024 took 1 seconds.
Segmenting frame-00025 to

### evaluation with metrics precision, recall & F1

In [12]:
# Single cell comparison
base_gt_mask = r"\\store\department\gene\chien_data\Lab\Data_and_Analysis\Wilco_van_Nes\Cell_tracking_challenge_datasets\Fluo-N2DL-HeLa\Fluo-N2DL-HeLa\02_ST\SEG"
base_pred_mask = r"\\store\department\gene\chien_data\Lab\Data_and_Analysis\Wilco_van_Nes\Cell_tracking_challenge_datasets\Fluo-N2DL-HeLa\Fluo-N2DL-HeLa\02_stardist"
frames_to_process = range(0, 84)
iou_threshold = 0.5

# --- Functions ---
def compute_iou(mask1, mask2):
    intersection = np.logical_and(mask1, mask2).sum()
    union = np.logical_or(mask1, mask2).sum()
    return intersection / union if union > 0 else 0

def match_objects(gt_mask, pred_mask, iou_threshold=0.5):
    gt_labels = label(gt_mask)
    pred_labels = label(pred_mask)

    gt_props = regionprops(gt_labels)
    pred_props = regionprops(pred_labels)

    iou_matrix = np.zeros((len(gt_props), len(pred_props)))

    for i, gt in enumerate(gt_props):
        for j, pred in enumerate(pred_props):
            iou = compute_iou(gt_labels == gt.label, pred_labels == pred.label)
            iou_matrix[i, j] = iou

    row_ind, col_ind = linear_sum_assignment(-iou_matrix)

    tp = 0
    fp = len(pred_props)
    fn = len(gt_props)
    ious_for_tp = []

    for i, j in zip(row_ind, col_ind):
        iou = iou_matrix[i, j]
        if iou >= iou_threshold:
            tp += 1
            fp -= 1
            fn -= 1
            ious_for_tp.append(iou)

    return tp, fp, fn, ious_for_tp

# --- Evaluation Loop ---
total_tp, total_fp, total_fn = 0, 0, 0
precision_list = []
recall_list = []
f1_list = []
all_ious = []

for frame in frames_to_process:
    fname_gt_mask = f"man_seg{frame:03d}.tif"
    fname_mask = f"t{frame:03d}.tif"

    fpath_gt = os.path.join(base_gt_mask, fname_gt_mask)
    fpath_pred = os.path.join(base_pred_mask, fname_mask)

    if not os.path.exists(fpath_gt) or not os.path.exists(fpath_pred):
        print(f"Skipping frame {frame:03d}: file not found.")
        continue

    gt_mask = cv2.imread(fpath_gt, -1)
    pred_mask = cv2.imread(fpath_pred, -1)

    tp, fp, fn, ious_for_tp = match_objects(gt_mask, pred_mask, iou_threshold=iou_threshold)
    total_tp += tp
    total_fp += fp
    total_fn += fn
    all_ious.extend(ious_for_tp)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)

    print(f"Frame {frame:03d} - TP: {tp}, FP: {fp}, FN: {fn} | Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")


# --- Final Metrics ---
total_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
total_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
total_f1 = 2 * total_precision * total_recall / (total_precision + total_recall) if (total_precision + total_recall) > 0 else 0

mean_precision = np.mean(precision_list) if precision_list else 0
mean_recall = np.mean(recall_list) if recall_list else 0
mean_f1 = np.mean(f1_list) if f1_list else 0
mean_iou = np.mean(all_ious) if all_ious else 0

print("\n--- Final Evaluation (Total Counts) ---")
print(f"Total True Positives: {total_tp}")
print(f"Total False Positives: {total_fp}")
print(f"Total False Negatives: {total_fn}")
print(f"Precision (Total): {total_precision:.4f}")
print(f"Recall (Total): {total_recall:.4f}")
print(f"F1 Score (Total): {total_f1:.4f}")
print(f"Average IoU (matched TPs): {mean_iou:.4f}")

print("\n--- Final Evaluation (Per-frame Average) ---")
print(f"Average Precision: {mean_precision:.4f}")
print(f"Average Recall: {mean_recall:.4f}")
print(f"Average F1 Score: {mean_f1:.4f}")

Frame 000 - TP: 117, FP: 30, FN: 7 | Precision: 0.7959, Recall: 0.9435, F1: 0.8635
Frame 001 - TP: 126, FP: 33, FN: 6 | Precision: 0.7925, Recall: 0.9545, F1: 0.8660
Frame 002 - TP: 128, FP: 38, FN: 6 | Precision: 0.7711, Recall: 0.9552, F1: 0.8533
Frame 003 - TP: 125, FP: 39, FN: 11 | Precision: 0.7622, Recall: 0.9191, F1: 0.8333
Frame 004 - TP: 135, FP: 28, FN: 8 | Precision: 0.8282, Recall: 0.9441, F1: 0.8824
Frame 005 - TP: 141, FP: 27, FN: 7 | Precision: 0.8393, Recall: 0.9527, F1: 0.8924
Frame 006 - TP: 148, FP: 25, FN: 8 | Precision: 0.8555, Recall: 0.9487, F1: 0.8997
Frame 007 - TP: 150, FP: 27, FN: 8 | Precision: 0.8475, Recall: 0.9494, F1: 0.8955
Frame 008 - TP: 156, FP: 27, FN: 8 | Precision: 0.8525, Recall: 0.9512, F1: 0.8991
Frame 009 - TP: 156, FP: 29, FN: 9 | Precision: 0.8432, Recall: 0.9455, F1: 0.8914
Frame 010 - TP: 160, FP: 24, FN: 7 | Precision: 0.8696, Recall: 0.9581, F1: 0.9117
Frame 011 - TP: 162, FP: 28, FN: 6 | Precision: 0.8526, Recall: 0.9643, F1: 0.9050
Fra

### Video creating code

In [14]:
# Paths
base_raw = r"\\store\department\gene\chien_data\Lab\Data_and_Analysis\Wilco_van_Nes\Cell_tracking_challenge_datasets\Fluo-N2DL-HeLa\Fluo-N2DL-HeLa\01"
base_mask = r"\\store\department\gene\chien_data\Lab\Data_and_Analysis\Wilco_van_Nes\Cell_tracking_challenge_datasets\Fluo-N2DL-HeLa\Fluo-N2DL-HeLa\01_stardist"
frames_to_process = range(0, 84)

# Output settings
output_path = r"\\store\department\gene\chien_data\Lab\Data_and_Analysis\Wilco_van_Nes\Cell_tracking_challenge_datasets\Fluo-N2DL-HeLa\Fluo-N2DL-HeLa\01_FL_stardist_contour.mp4"
fps = 10
alpha = 0.5  # Transparency for overlay

# Generate a colormap: label_id -> color
def generate_label_colormap(max_labels=256):
    random.seed(42)  # For reproducibility
    return {
        label: tuple(random.randint(0, 255) for _ in range(3))
        for label in range(1, max_labels)
    }

label_colors = generate_label_colormap()

# Get frame size from first valid raw image
first_valid = None
for frame in frames_to_process:
    fpath_raw = os.path.join(base_raw, f"t{frame:03d}.tif")
    if os.path.exists(fpath_raw):
        first_valid = cv2.imread(fpath_raw, -1)
        break

if first_valid is None:
    raise RuntimeError("No valid frames found.")

height, width = first_valid.shape
video_writer = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height), isColor=True)

for frame in frames_to_process:
    fname_raw = f"t{frame:03d}.tif"
    fname_mask = f"t{frame:03d}.tif"
    fpath_raw = os.path.join(base_raw, fname_raw)
    fpath_mask = os.path.join(base_mask, fname_mask)

    if not os.path.exists(fpath_raw) or not os.path.exists(fpath_mask):
        print(f"Skipping frame {frame:03d}: file not found.")
        continue

    # Load raw and mask
    raw_img = cv2.imread(fpath_raw, -1)
    mask_img = cv2.imread(fpath_mask, -1)

    # Normalize raw image to uint8
    raw_img = cv2.normalize(raw_img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # Convert single-channel grayscale to 3-channel BGR manually
    raw_bgr = cv2.merge([
    np.zeros_like(raw_img),  # Blue channel
    np.zeros_like(raw_img),  # Green channel
    raw_img                  # Red channel
    ])

    # Create a blank color image for the mask overlay
    mask_color = np.zeros_like(raw_bgr)

    # Draw each label in its own color
    unique_labels = np.unique(mask_img)
    for label in unique_labels:
        if label == 0:
            continue  # skip background

        color = label_colors.get(label % 255, (255, 255, 255))  # prevent missing keys
        mask = (mask_img == label)
        mask_color[mask] = color

    # mask_binary = (mask_img > 0)
    # mask_color[mask_binary] = (255, 255, 255)

    # Blend raw and color mask
    overlay = cv2.addWeighted(raw_bgr, 1 - alpha, mask_color, alpha, 0)

    # Write to video
    video_writer.write(overlay)

video_writer.release()
print(f"Colored overlay video saved to: {output_path}")

Colored overlay video saved to: \\store\department\gene\chien_data\Lab\Data_and_Analysis\Wilco_van_Nes\Cell_tracking_challenge_datasets\Fluo-N2DL-HeLa\Fluo-N2DL-HeLa\01_FL_stardist_contour.mp4


## For single file only

In [4]:
fluorescent_raw_img = cv2.imread(r"\\store\department\gene\chien_data\Lab\Data_and_Analysis\Tsai-Ying_Chen\TYC069_EC546_TILs_48hr_20250312\48hr_record__20250312at111801\460nm\48hr_record__50B_1500ms_frame00000_ROI10.tif", -1)

fluorescent_norm_img = normalize(fluorescent_raw_img, 1, 99.8, axis=(0, 1))

# fl_labels, polys = model.predict_instances(fluorescent_norm_img, prob_thresh = prob_thresh)
fl_labels, polys = model.predict_instances(fluorescent_norm_img, prob_thresh = prob_thresh)

# --- Save nucleus segmentation ---
# out1 = os.path.join(out_nucleus, fname_raw)
cv2.imwrite(r"\\store\department\gene\chien_data\Lab\Labmembers\Wilco_van_Nes\Thesis\Figures\Experiment 4C\48hr_record__50B_1500ms_frame00000_ROI10_seg_random5.tif", fl_labels.astype(np.uint16))

: 